<a href="https://colab.research.google.com/github/cheaqingqing/ADSP/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install openpyxl

In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/ADSP Final Project/students_data.xlsx"

In [31]:
class Node:
    def __init__(self, student_id, first_name, last_name, gpa):
        self.student_id = student_id
        self.first_name = first_name
        self.last_name = last_name
        self.gpa = gpa
        self.left = None
        self.right = None

In [42]:
class BST:
    def __init__(self, order_by="id"):
        self.root = None
        # order_by can be "id" or "gpa"
        self.order_by = order_by

    def get_key(self, node):
        if self.order_by == "id":
            return node.student_id
        elif self.order_by == "gpa":
            return node.gpa

    def insert(self, root, student_id, first_name, last_name, gpa):
        if root is None:
            return Node(student_id, first_name, last_name, gpa)
        if student_id < root.student_id:
            root.left = self.insert(root.left, student_id, first_name, last_name, gpa)
        elif student_id > root.student_id:
            root.right = self.insert(root.right, student_id, first_name, last_name, gpa)
        return root

    def inorder(self, root):
        if root:
            self.inorder(root.left)
            print(f"ID:{root.student_id}, Name:{root.first_name} {root.last_name}, GPA:{root.gpa}")
            self.inorder(root.right)

    def search(self, root, key_value):
        if root is None:
            return None
        if key_value == self.get_key(root):
            return root
        elif key_value < self.get_key(root):
            return self.search(root.left, key_value)
        else:
            return self.search(root.right, key_value)

    def search_by_gpa(self, root, target_gpa, results=None):
      if results is None:
          results = []
      if root:
          self.search_by_gpa(root.left, target_gpa, results)
          if root.gpa == target_gpa:
              results.append(root)
          self.search_by_gpa(root.right, target_gpa, results)
      return results

    def delete(self, root, student_id):
        if root is None:
            return root
        if student_id < root.student_id:
            root.left = self.delete(root.left, student_id)
        elif student_id > root.student_id:
            root.right = self.delete(root.right, student_id)
        else:
            # Node found
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            # Find inorder successor
            temp = self.minValueNode(root.right)
            root.student_id, root.first_name, root.last_name, root.gpa = (
                temp.student_id, temp.first_name, temp.last_name, temp.gpa
            )
            root.right = self.delete(root.right, temp.student_id)
        return root

    def minValueNode(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

In [33]:
try:
    students_data = pd.read_excel(file_path)
except FileNotFoundError:
    students_data = pd.DataFrame(columns=["Student ID", "First_Name", "Last_Name", "GPA"])

In [36]:
bst = BST()
root = None

# Insert existing records into BST
for _, row in students_data.iterrows():
    root = bst.insert(root, (row["Student ID"]), row["First Name"], row["Last Name"], float(row["GPA"]))

In [44]:
while True:
    print("\n===== Student BST Menu =====")
    print("1. Display Info")
    print("2. Add Info")
    print("3. Remove Info")
    print("4. Search Info")
    print("5. Exit")

    choice = input("Enter choice: ")

    if choice == "1":
        print("\n--- Student Records ---")
        bst.inorder(root)


    elif choice == "2":
        sid = int(input("Enter Student ID: "))
        fname = input("Enter First Name: ")
        lname = input("Enter Last Name: ")
        gpa_ = float(input("Enter GPA: "))
        root = bst.insert(root, sid, fname, lname, gpa_)
        students_data = pd.concat([students_data, pd.DataFrame([
            {"Student_ID": sid, "First_Name": fname, "Last_Name": lname, "GPA": gpa_}
        ])], ignore_index=True)
        students_data.to_excel(file_path, index=False)
        print("✅ Student added successfully!")

    elif choice == "3":
        sid = int(input("Enter Student ID to remove: "))
        root = bst.delete(root, sid)
        students_data = students_data[students_data["Student_ID"] != sid]
        students_data.to_excel(file_path, index=False)
        print("✅ Student removed successfully!")

    elif choice == "4":
      print("\nSearch by:")
      print("1. Student ID")
      print("2. GPA")
      sub_choice = input("Enter choice: ")

      if sub_choice == "1":
          sid = int(input("Enter Student ID to search: "))
          node = bst.search(root, sid)
          if node:
              print("✅ Student found:")
              print(f"ID:{node.student_id}, Name:{node.first_name} {node.last_name}, GPA:{node.gpa}")
          else:
              print("❌ Student not found!")

      elif sub_choice == "2":
          gpa_target = float(input("Enter GPA to search: "))
          matches = [] # Initialize an empty list to store results
          bst.search_by_gpa(root, gpa_target, matches) # Pass the list to the method
          if matches:
              print(f"✅ Found {len(matches)} student(s) with GPA {gpa_target}:")
              for node in matches:
                  print(f"ID:{node.student_id}, Name:{node.first_name} {node.last_name}, GPA:{node.gpa}")
          else:
              print("❌ No students found with that GPA.")

    elif choice == "5":
        print("Exiting program...")
        break
    else:
        print("Invalid choice, try again.")


===== Student BST Menu =====
1. Display Info
2. Add Info
3. Remove Info
4. Search Info
5. Exit
Enter choice: 4

Search by:
1. Student ID
2. GPA
Enter choice: 2
Enter GPA to search: 3.25
✅ Found 2 student(s) with GPA 3.25:
ID:18.0, Name:VONG SOKMEAN, GPA:3.25
ID:22.0, Name:SRUN SOCHETTRA, GPA:3.25

===== Student BST Menu =====
1. Display Info
2. Add Info
3. Remove Info
4. Search Info
5. Exit
Enter choice: 5
Exiting program...
